<a href="https://colab.research.google.com/github/Face-Expression-Recognation-Team-2/Face-Emotion-Recognation/blob/main/Face_Emotion_Recognition_(_Team_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Memasukkan Library

In [ ]:
import numpy as np
import pandas as pd
import os
import zipfile
import shutil

#Memasukkan data

In [ ]:
!pip install -q kaggle
! mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d 'jonathanoheix/face-expression-recognition-dataset'

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 89% 107M/121M [00:00<00:00, 232MB/s] 
100% 121M/121M [00:00<00:00, 208MB/s]


In [ ]:
dataset_zip = zipfile.ZipFile('face-expression-recognition-dataset.zip', 'r')

dataset_zip.extractall()

dataset_zip.close()

In [ ]:
source_folders = ['/content/images/images/train', '/content/images/images/validation']
destination_folder = '/content/dataset'

if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for folder in source_folders:
    shutil.move(folder, destination_folder)
    print(f"Folder {folder} berhasil dipindahkan ke {destination_folder}")

print("Success")

Folder /content/images/images/train berhasil dipindahkan ke /content/dataset
Folder /content/images/images/validation berhasil dipindahkan ke /content/dataset
Success


In [ ]:
old_folder = '/content/images'

shutil.rmtree(old_folder)

print("Success")

Success
